In [ ]:

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

df = pd.read_csv("../data/heart_disease.csv")
X = df.drop(columns=["target"])
num = X.select_dtypes("number").columns.tolist()
cat = [c for c in X.columns if c not in num]

pre = ColumnTransformer([
    ("num", Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]), num),
    ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("ohe", OneHotEncoder(handle_unknown="ignore"))]), cat)
])
Xpre = pre.fit_transform(X)
km = KMeans(n_clusters=2, random_state=42, n_init=10).fit(Xpre)
dict((i, int((km.labels_==i).sum())) for i in range(2))
